In [17]:
import os
import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [18]:
root = '../../Assets/101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

['../../Assets/101_ObjectCategories\\accordion', '../../Assets/101_ObjectCategories\\anchor', '../../Assets/101_ObjectCategories\\ant', '../../Assets/101_ObjectCategories\\barrel', '../../Assets/101_ObjectCategories\\bass', '../../Assets/101_ObjectCategories\\beaver', '../../Assets/101_ObjectCategories\\binocular', '../../Assets/101_ObjectCategories\\bonsai', '../../Assets/101_ObjectCategories\\brain', '../../Assets/101_ObjectCategories\\brontosaurus', '../../Assets/101_ObjectCategories\\buddha', '../../Assets/101_ObjectCategories\\butterfly', '../../Assets/101_ObjectCategories\\camera', '../../Assets/101_ObjectCategories\\cannon', '../../Assets/101_ObjectCategories\\car_side', '../../Assets/101_ObjectCategories\\ceiling_fan', '../../Assets/101_ObjectCategories\\cellphone', '../../Assets/101_ObjectCategories\\chair', '../../Assets/101_ObjectCategories\\chandelier', '../../Assets/101_ObjectCategories\\cougar_body', '../../Assets/101_ObjectCategories\\cougar_face', '../../Assets/101_Obje

In [21]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [24]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

In [25]:
data[0]

{'x': array([[[-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         ...,
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ]],
 
        [[-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         ...,
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ]],
 
        [[-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         ...,
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ]],
 
        ...,
 
        [[-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         ...,
         [-103.939, -116.779, -123.68 ],
         [-10